# Implementação AWS Elastic Beanstalk
Sequencia para criação de um serviço utilziando boto3

In [1]:
try:
    !pip install boto3=="1.13.1" --quiet
except:
    print("Running throw py file.")

In [2]:
import boto3
import os
import subprocess
import json

## Preparando os serviços AWS Elastic Beanstalk
Utilizando o doc https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/elasticbeanstalk.html como apoio

In [4]:
bs_client = boto3.client('elasticbeanstalk')

In [6]:
appName = 'CasaDoCodigoApp'
appEnv = 'CasaDoCodigoEnv'
appKey = 'CasaDoCodigo'
my_tag = [{'Key': appKey, 'Value': 'string'}]

Criando uma aplicação Elastic Beanstalk

In [7]:
bs_client.create_application(
    ApplicationName = appName,
    Description = 'Exemplo Alura de utilização de Elastic Beanstalk',
    Tags = my_tag
)

{'Application': {'ApplicationArn': 'arn:aws:elasticbeanstalk:us-east-1:032594213725:application/CasaDoCodigoApp',
  'ApplicationName': 'CasaDoCodigoApp',
  'Description': 'Exemplo Alura de utilização de Elastic Beanstalk',
  'DateCreated': datetime.datetime(2020, 5, 24, 3, 2, 1, 785000, tzinfo=tzutc()),
  'DateUpdated': datetime.datetime(2020, 5, 24, 3, 2, 1, 785000, tzinfo=tzutc()),
  'ConfigurationTemplates': [],
  'ResourceLifecycleConfig': {'VersionLifecycleConfig': {'MaxCountRule': {'Enabled': False,
     'MaxCount': 200,
     'DeleteSourceFromS3': False},
    'MaxAgeRule': {'Enabled': False,
     'MaxAgeInDays': 180,
     'DeleteSourceFromS3': False}}}},
 'ResponseMetadata': {'RequestId': '0dd6be9d-1a1d-4710-8f73-e28a40511bff',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml',
   'date': 'Sun, 24 May 2020 03:02:01 GMT',
   'x-amzn-requestid': '0dd6be9d-1a1d-4710-8f73-e28a40511bff',
   'content-length': '1229',
   'connection': 'keep-alive'},
  'RetryAttempts'

Identificando o Bucket criado

In [11]:
my_bucket = bs_client.create_storage_location()['S3Bucket']
my_bucket

'elasticbeanstalk-us-east-1-032594213725'

Retorna as descricoes de aplicacoes existentes.

In [12]:
bs_app = bs_client.describe_applications(
    ApplicationNames=[ appName, ])
bs_app

{'Applications': [{'ApplicationArn': 'arn:aws:elasticbeanstalk:us-east-1:032594213725:application/CasaDoCodigoApp',
   'ApplicationName': 'CasaDoCodigoApp',
   'Description': 'Exemplo Alura de utilização de Elastic Beanstalk',
   'DateCreated': datetime.datetime(2020, 5, 24, 3, 2, 1, 785000, tzinfo=tzutc()),
   'DateUpdated': datetime.datetime(2020, 5, 24, 3, 2, 1, 785000, tzinfo=tzutc()),
   'ConfigurationTemplates': [],
   'ResourceLifecycleConfig': {'VersionLifecycleConfig': {'MaxCountRule': {'Enabled': False,
      'MaxCount': 200,
      'DeleteSourceFromS3': False},
     'MaxAgeRule': {'Enabled': False,
      'MaxAgeInDays': 180,
      'DeleteSourceFromS3': False}}}}],
 'ResponseMetadata': {'RequestId': 'cc78dd8c-5adc-4561-adff-fadf0d826095',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml',
   'date': 'Sun, 24 May 2020 03:21:58 GMT',
   'x-amzn-requestid': 'cc78dd8c-5adc-4561-adff-fadf0d826095',
   'content-length': '1314',
   'connection': 'keep-alive'},
  '

In [13]:
ApplicationArn = bs_app['Applications'][0]['ApplicationArn']
ApplicationArn

'arn:aws:elasticbeanstalk:us-east-1:032594213725:application/CasaDoCodigoApp'

### Criar arquivo WAR da aplicação JAVA

In [14]:
def run_command(command):
    p = subprocess.Popen(command,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.STDOUT)
    return iter(p.stdout.readline, b'')

In [15]:
for output_line in run_command('jar -cvf ROOT.war casadocodigo/*'):
    print(output_line)

b'added manifest\r\n'
b'adding: casadocodigo/.classpath(in = 1148) (out= 348)(deflated 69%)\r\n'
b'adding: casadocodigo/.gitignore(in = 49) (out= 51)(deflated -4%)\r\n'
b'adding: casadocodigo/.project(in = 1088) (out= 301)(deflated 72%)\r\n'
b'adding: casadocodigo/.settings/(in = 0) (out= 0)(stored 0%)\r\n'
b'adding: casadocodigo/.settings/.jsdtscope(in = 639) (out= 288)(deflated 54%)\r\n'
b'adding: casadocodigo/.settings/org.eclipse.core.resources.prefs(in = 119) (out= 80)(deflated 32%)\r\n'
b'adding: casadocodigo/.settings/org.eclipse.jdt.core.prefs(in = 616) (out= 193)(deflated 68%)\r\n'
b'adding: casadocodigo/.settings/org.eclipse.m2e.core.prefs(in = 90) (out= 75)(deflated 16%)\r\n'
b'adding: casadocodigo/.settings/org.eclipse.wst.common.component(in = 677) (out= 266)(deflated 60%)\r\n'
b'adding: casadocodigo/.settings/org.eclipse.wst.common.project.facet.core.xml(in = 252) (out= 130)(deflated 48%)\r\n'
b'adding: casadocodigo/.settings/org.eclipse.wst.jsdt.ui.superType.container(in

## Upload to Bucket S3

In [37]:
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

# Listando todos os Buckets
for bucket in s3.buckets.all():
    print(bucket.name)

elasticbeanstalk-us-east-1-032594213725
estudo-aws-fabio
estudo-aws-fk


Upload do arquivo WAR para o Bucket

In [25]:
files = ['ROOT.war']
dataPath = os.getcwd()

for file in files: 
    file_name = dataPath + "/" + file
    try:
        response = s3_client.upload_file(file_name, my_bucket, "app/" + file)
        print("It was uploaded the file", "'" + file + "'.")
    except ClientError as e:
        logging.error(e)

It was uploaded the file 'ROOT.war' .


Criando uma TAG no Bucket Criado

In [26]:
tag={'TagSet': my_tag}
s3_client.put_bucket_tagging(Bucket=my_bucket, Tagging=tag)

{'ResponseMetadata': {'RequestId': '67B4107AA3965E3D',
  'HostId': 'UJ095K1DGmN4pqXF7pvdSs2jcW6Y+ZADVX2xoO8CvxZ6ODeBDmBcoHWHTQrcsm59eAdXE0EZkRw=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'UJ095K1DGmN4pqXF7pvdSs2jcW6Y+ZADVX2xoO8CvxZ6ODeBDmBcoHWHTQrcsm59eAdXE0EZkRw=',
   'x-amz-request-id': '67B4107AA3965E3D',
   'date': 'Sun, 24 May 2020 03:36:30 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

## Beanstalk Create Application Version
Criando uma Application Version. Aqui que envio o arquivo WAR.

In [33]:
my_version = 'v1'

bs_client.create_application_version(
    ApplicationName=appName,
    VersionLabel=my_version,
    Description='string',
    SourceBundle={
        'S3Bucket': my_bucket,
        'S3Key': 'app/ROOT.war'
    },
    Process=True,
)

{'ApplicationVersion': {'ApplicationVersionArn': 'arn:aws:elasticbeanstalk:us-east-1:032594213725:applicationversion/CasaDoCodigoApp/v1',
  'ApplicationName': 'CasaDoCodigoApp',
  'Description': 'string',
  'VersionLabel': 'v1',
  'SourceBundle': {'S3Bucket': 'elasticbeanstalk-us-east-1-032594213725',
   'S3Key': 'app/ROOT.war'},
  'DateCreated': datetime.datetime(2020, 5, 24, 3, 42, 15, 919000, tzinfo=tzutc()),
  'DateUpdated': datetime.datetime(2020, 5, 24, 3, 42, 15, 919000, tzinfo=tzutc()),
  'Status': 'PROCESSING'},
 'ResponseMetadata': {'RequestId': '8f824e01-0157-41c2-b3ca-d01592b79a6a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml',
   'date': 'Sun, 24 May 2020 03:42:15 GMT',
   'x-amzn-requestid': '8f824e01-0157-41c2-b3ca-d01592b79a6a',
   'content-length': '935',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

Lista dos nomes de soluções AWS

In [35]:
bs_client.list_available_solution_stacks()['SolutionStacks']


['64bit Windows Server Core 2019 v2.5.6 running IIS 10.0',
 '64bit Windows Server 2019 v2.5.6 running IIS 10.0',
 '64bit Windows Server Core 2016 v2.5.6 running IIS 10.0',
 '64bit Windows Server 2016 v2.5.6 running IIS 10.0',
 '64bit Windows Server Core 2012 R2 v2.5.6 running IIS 8.5',
 '64bit Windows Server 2012 R2 v2.5.6 running IIS 8.5',
 '64bit Windows Server Core 2016 v1.2.0 running IIS 10.0',
 '64bit Windows Server 2016 v1.2.0 running IIS 10.0',
 '64bit Windows Server Core 2012 R2 v1.2.0 running IIS 8.5',
 '64bit Windows Server 2012 R2 v1.2.0 running IIS 8.5',
 '64bit Windows Server 2012 v1.2.0 running IIS 8',
 '64bit Amazon Linux 2018.03 v2.15.0 running GlassFish 5.0 Java 8 (Preconfigured - Docker)',
 '64bit Amazon Linux 2018.03 v2.10.7 running Java 8',
 '64bit Amazon Linux 2018.03 v2.10.7 running Java 7',
 '64bit Amazon Linux 2018.03 v4.14.3 running Node.js',
 '64bit Amazon Linux 2018.03 v2.9.6 running PHP 5.4',
 '64bit Amazon Linux 2018.03 v2.9.6 running PHP 5.5',
 '64bit Amaz

In [36]:
bs_client.create_environment(
    ApplicationName = appName,
    EnvironmentName = appEnv,
    CNAMEPrefix = appName,
    Tags= my_tag,

    SolutionStackName='64bit Amazon Linux 2018.03 v3.3.6 running Tomcat 8.5 Java 8'
)

{'EnvironmentName': 'CasaDoCodigoEnv',
 'EnvironmentId': 'e-n7k3mtrmcw',
 'ApplicationName': 'CasaDoCodigoApp',
 'SolutionStackName': '64bit Amazon Linux 2018.03 v3.3.6 running Tomcat 8.5 Java 8',
 'PlatformArn': 'arn:aws:elasticbeanstalk:us-east-1::platform/Tomcat 8.5 with Java 8 running on 64bit Amazon Linux/3.3.6',
 'CNAME': 'CasaDoCodigoApp.us-east-1.elasticbeanstalk.com',
 'DateCreated': datetime.datetime(2020, 5, 24, 3, 45, 51, 258000, tzinfo=tzutc()),
 'DateUpdated': datetime.datetime(2020, 5, 24, 3, 45, 51, 258000, tzinfo=tzutc()),
 'Status': 'Launching',
 'Health': 'Grey',
 'Tier': {'Name': 'WebServer', 'Type': 'Standard', 'Version': '1.0'},
 'EnvironmentArn': 'arn:aws:elasticbeanstalk:us-east-1:032594213725:environment/CasaDoCodigoApp/CasaDoCodigoEnv',
 'ResponseMetadata': {'RequestId': '3f15507f-bd18-4411-8060-9cef32598bc9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml',
   'date': 'Sun, 24 May 2020 03:45:51 GMT',
   'x-amzn-requestid': '3f15507f-bd18

Neste momento é criado o ambiente. A AWS criara uma instancia EC2

In [38]:
bs_client.update_environment(
            ApplicationName = appName,
            EnvironmentName = appEnv,
            VersionLabel=my_version,
        )

{'EnvironmentName': 'CasaDoCodigoEnv',
 'EnvironmentId': 'e-n7k3mtrmcw',
 'ApplicationName': 'CasaDoCodigoApp',
 'VersionLabel': 'v1',
 'SolutionStackName': '64bit Amazon Linux 2018.03 v3.3.6 running Tomcat 8.5 Java 8',
 'PlatformArn': 'arn:aws:elasticbeanstalk:us-east-1::platform/Tomcat 8.5 with Java 8 running on 64bit Amazon Linux/3.3.6',
 'EndpointURL': 'awseb-e-n-AWSEBLoa-TAI4JZV8SQQU-803235371.us-east-1.elb.amazonaws.com',
 'CNAME': 'CasaDoCodigoApp.us-east-1.elasticbeanstalk.com',
 'DateCreated': datetime.datetime(2020, 5, 24, 3, 45, 51, 226000, tzinfo=tzutc()),
 'DateUpdated': datetime.datetime(2020, 5, 24, 3, 54, 29, 311000, tzinfo=tzutc()),
 'Status': 'Updating',
 'AbortableOperationInProgress': True,
 'Health': 'Grey',
 'Tier': {'Name': 'WebServer', 'Type': 'Standard', 'Version': '1.0'},
 'EnvironmentArn': 'arn:aws:elasticbeanstalk:us-east-1:032594213725:environment/CasaDoCodigoApp/CasaDoCodigoEnv',
 'ResponseMetadata': {'RequestId': '70949217-5533-4dd7-982d-aa67cc27dea4',
  

In [40]:
bs_client.describe_application_versions(
    ApplicationName=appName,
    VersionLabels=[
        my_version,
    ],
)

{'ApplicationVersions': [{'ApplicationVersionArn': 'arn:aws:elasticbeanstalk:us-east-1:032594213725:applicationversion/CasaDoCodigoApp/v1',
   'ApplicationName': 'CasaDoCodigoApp',
   'Description': 'string',
   'VersionLabel': 'v1',
   'SourceBundle': {'S3Bucket': 'elasticbeanstalk-us-east-1-032594213725',
    'S3Key': 'app/ROOT.war'},
   'DateCreated': datetime.datetime(2020, 5, 24, 3, 42, 15, 919000, tzinfo=tzutc()),
   'DateUpdated': datetime.datetime(2020, 5, 24, 3, 42, 15, 919000, tzinfo=tzutc()),
   'Status': 'PROCESSED'}],
 'ResponseMetadata': {'RequestId': '05525902-0342-4e77-a5c8-541f66417999',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml',
   'date': 'Sun, 24 May 2020 04:11:45 GMT',
   'x-amzn-requestid': '05525902-0342-4e77-a5c8-541f66417999',
   'content-length': '1001',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [44]:
appEnv

'CasaDoCodigoEnv'

In [43]:
appName

'CasaDoCodigoApp'

## Remove Aplicação Beanstalk

Remove o enviroment configurations

In [42]:
bs_client.terminate_environment(
    ApplicationName=appName,
    EnvironmentName=appEnv,
)

ParamValidationError: Parameter validation failed:
Unknown parameter in input: "ApplicationName", must be one of: EnvironmentId, EnvironmentName, TerminateResources, ForceTerminate

In [54]:
bs_client.delete_environment_configuration(
    ApplicationName=appName,
    EnvironmentName=appEnv
)

ClientError: An error occurred (InvalidParameterValue) when calling the DeleteEnvironmentConfiguration operation: Cannot delete the environment for application named CasaDoCodigoApp. The application is currently pending deletion.

Remove o application version

In [55]:
bs_client.delete_application_version(
    ApplicationName=appName,
    VersionLabel=my_version,
    DeleteSourceBundle=False
)

ClientError: An error occurred (InvalidParameterValue) when calling the DeleteApplicationVersion operation: Cannot delete the version for application named CasaDoCodigoApp. The application is currently pending deletion.

In [56]:
bs_client.delete_application(
    ApplicationName=appName,
    TerminateEnvByForce=True
)

{'ResponseMetadata': {'RequestId': '3ef77de9-214d-4f4e-98a7-67671cc348fa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml',
   'date': 'Sun, 24 May 2020 04:18:25 GMT',
   'x-amzn-requestid': '3ef77de9-214d-4f4e-98a7-67671cc348fa',
   'content-length': '227',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Limpando e removendo o Bucket

In [57]:
bucket = s3.Bucket(my_bucket)
bucket.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '34B361DC2F797880',
   'HostId': 'RHtEaAI8Z0NF9jM5rR7j9wc8tAossCCZD4yxMPWwwkzIyiuzCy1G6r6Pc3IO//1ak2EdeRVloyA=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'RHtEaAI8Z0NF9jM5rR7j9wc8tAossCCZD4yxMPWwwkzIyiuzCy1G6r6Pc3IO//1ak2EdeRVloyA=',
    'x-amz-request-id': '34B361DC2F797880',
    'date': 'Sun, 24 May 2020 04:18:58 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'resources/_runtime/_embedded_extensions/CasaDoCodigoApp/3239e60139db014506a1b3a1f791a418'},
   {'Key': 'resources/environments/e-n7k3mtrmcw/_runtime/versions/manifest_1590293188087'},
   {'Key': 'resources/environments/e-n7k3mtrmcw/_runtime/versions/manifest_1590292982163'},
   {'Key': 'resources/environments/e-n7k3mtrmcw/_runtime/versions/manifest_1590291960521'},
   {'Key': 'resources/environments/e-n7k3mtrmcw/_runtime/versions/manifest_1590

In [60]:
s3_client.delete_bucket(Bucket=my_bucket)

ClientError: An error occurred (AccessDenied) when calling the DeleteBucket operation: Access Denied